In [22]:
import requests
import json
import sqlite3
from urllib.parse import unquote

# Request
### request로 open AI를 이용하려고 했으나 csv 파일로 처리가 가능할 것 같아서 사용하지 않음

In [77]:
API_KEY = unquote('yTgRonec4t9G1QvTLdIZnqJjoqI4q%2BAh5D6%2BofqrynIU%2BUqHt6Rn6DJovgZRFa6ghJXFeYoLGXczD%2ByXOFajYQ%3D%3D')
url = 'http://api.data.go.kr/openapi/tn_pubr_public_cltur_fstvl_api'
# params ={'serviceKey' : API_KEY, 
#          'pageNo' : '1', 
#          'numOfRows' : '100', 
#          'type' : 'json', 
#          'fstvlNm' : '서면 드림 축제', 
#          'opar' : '',
#          'fstvlStartDate' : '', 
#          'fstvlEndDate' : '', 
#          'fstvlCo' : '',
#          'mnnstNm' : '', 
#          'auspcInsttNm' : '', 
#          'suprtInsttNm' : '', 
#          'phoneNumber' : '', 
#          'homepageUrl' : '', 
#          'relateInfo' : '',
#          'rdnmadr' : '',
#          'lnmadr' : '', 
#          'latitude' : '', 
#          'longitude' : '', 
#          'referenceDate' : '', 
#          'instt_code' : '',
#          'instt_nm' : '' }

params = {'serviceKey' : API_KEY, 
         'pageNo' : '1', 
         'numOfRows' : '100', 
         'type' : 'json',
          'fstvlNM' : '서면 드림 축제'}

#아하.. 공백일 땐 아예 params에서 빼야 정상적으로 실행되네...

response = requests.get(url, params=params)
print(response.json())



# Geocode

In [38]:
def getGeoCode(address, client_id, client_secret):
    header={
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }
    endpoint="https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    url = f"{endpoint}?query={address}"
    
    res = requests.get(url, headers=header)
    
    return res



In [88]:
def geoCode(adr, client_id, client_secret):
    address = adr
    
    response = getGeoCode(address, client_id, client_secret)
   
    if(response.status_code == 200):
            result = response.json()
            result_lst = [result['addresses'][0]['x'], result['addresses'][0]['y']]
            return result_lst
    else:
            print(f'Error code: {response}')

## 데이터 전처리: geoCode()로 위도와 경도 값을 구한다.
### 모든 튜플에는 도로명주소(11열) 또는 지번주소(12열)이 적혀있다.
### step 1. 도로명 주소만 존재하는데이터 처리

In [44]:
import csv
#csv 파일 열기
f = open('cultureFestival.csv', 'r', newline='')
reader = csv.reader(f)
gis =["5twl61m1m1", "DtCxHuTaLRrjAkvIOXQPJTrBNP2CAngMOevgmJ2a"] #gis[0] = geoCode id, gis[1] = geoCode secret
rows = list(reader)
f.close()

for i in range(1, len(rows)):
    if rows[i][11] != '' and rows[i][13] == '' and rows[i][14] == '': #도로명 주소가 주어졌지만 위도, 경도는 주어지지 않은 경우
        try:
            result = geoCode(rows[i][11], gis[0], gis[1]) #result[0]은 경도, result[1]은 위도
            rows[i][14] = result[0]
            rows[i][13] = result[1]
        except:
            continue
f.close()

0.568098783493042


### step2. 지번주소만 존재하는 데이터 처리

In [52]:
import csv
import time
start_time = time.time()
#csv 파일 열기
f = open('cultureFestival.csv', 'r', newline='')
reader = csv.reader(f)
gis =["5twl61m1m1", "DtCxHuTaLRrjAkvIOXQPJTrBNP2CAngMOevgmJ2a"] #gis[0] = geoCode id, gis[1] = geoCode secret
rows = list(reader)
f.close()

for i in range(1, len(rows)):
    if rows[i][12] != '' and rows[i][13] == '' and rows[i][14] == '': #도로명 주소가 주어졌지만 위도, 경도는 주어지지 않은 경우
        try:
            result = geoCode(rows[i][12], gis[0], gis[1]) #result[0]은 경도, result[1]은 위도
            rows[i][14] = result[0]
            rows[i][13] = result[1]
        except:
            continue


f = open('cultureFestival.csv', 'w', newline='')
writer = csv.writer(f)
writer.writerows(rows)
f.close()

print(time.time() -start_time)

6.544418811798096


### 아직도 처리되지 않은 데이터가 있는지 확인

In [53]:
for i in range(len(rows)):
    if rows[i][13] == '' or rows[i][14] == '':
        print(i+1, rows[i][0])
#202, 613, 1322, 1325행의 데이터들이 처리되지 않았다.        

202 지봉로 상사화 축제
613 대왕암 해맞이 축제
1322 서리풀페스티벌
1325 악기거리축제


In [59]:
print('202: ', rows[202][11], '613: ',rows[613][11], '1322: ',rows[1322][11], '1325: ',rows[1325][11], rows[1325][12])
#202, 613, 1322행은 도로명 주소, 1325행은 지번주소만 나와있다.

202:  전라남도 순천시 장명로 30 613:  울산광역시 동구 문현3길 6 1322:  서울특별시 서초구 남부순환로356길 15 1325:   서울특별시 서초구 잠원동 58-14


In [64]:
#왜 처리가 안 됐는진 모르겠으나, 처리하기 위해 geoCode의 리턴값을 수정해서 데이터에 이상이 있는지 확인한다. -> 이상 없음
def geoCode(adr, client_id, client_secret):
    address = adr
    
    response = getGeoCode(address, client_id, client_secret)
   
    if(response.status_code == 200):
            result = response.json()
            return result
    else:
            print(f'Error code: {response}')
result1 = geoCode('전라남도 순천시 장명로 30', gis[0], gis[1])
print(result1)
print()

result2 = geoCode('울산광역시 동구 문현3길 6', gis[0], gis[1])
print(result2)
print()

result3 = geoCode('서울특별시 서초구 남부순환로356길 15', gis[0], gis[1])
print(result3)
print()

result4 = geoCode('서울특별시 서초구 잠원동 58-14', gis[0], gis[1])
print(result4)

{'status': 'OK', 'meta': {'totalCount': 1, 'page': 1, 'count': 1}, 'addresses': [{'roadAddress': '전라남도 순천시 장명로 30 순천시청', 'jibunAddress': '전라남도 순천시 장천동 53-1 순천시청', 'englishAddress': '30, Jangmyeong-ro, Suncheon-si, Jeollanam-do, Republic of Korea', 'addressElements': [{'types': ['SIDO'], 'longName': '전라남도', 'shortName': '전라남도', 'code': ''}, {'types': ['SIGUGUN'], 'longName': '순천시', 'shortName': '순천시', 'code': ''}, {'types': ['DONGMYUN'], 'longName': '장천동', 'shortName': '장천동', 'code': ''}, {'types': ['RI'], 'longName': '', 'shortName': '', 'code': ''}, {'types': ['ROAD_NAME'], 'longName': '장명로', 'shortName': '장명로', 'code': ''}, {'types': ['BUILDING_NUMBER'], 'longName': '30', 'shortName': '30', 'code': ''}, {'types': ['BUILDING_NAME'], 'longName': '순천시청', 'shortName': '순천시청', 'code': ''}, {'types': ['LAND_NUMBER'], 'longName': '53-1', 'shortName': '53-1', 'code': ''}, {'types': ['POSTAL_CODE'], 'longName': '57956', 'shortName': '57956', 'code': ''}], 'x': '127.4872430', 'y': '34.9506984'

In [68]:
#result_lst = [result['addresses'][0]['x'], result['addresses'][0]['y']]
print(result1['addresses'][0]['x'], result1['addresses'][0]['y'])
print(result2['addresses'][0]['x'], result2['addresses'][0]['y'])
print(result3['addresses'][0]['x'], result3['addresses'][0]['y'])
print(result4['addresses'][0]['x'], result4['addresses'][0]['y'])

127.4872430 34.9506984
129.4112834 35.4870810
127.0379729 37.4843263
127.0142882 37.5142565


In [89]:
#직접 추가 후 다시 csv 파일에 작성
rows[201][14] = result1['addresses'][0]['x']
rows[201][13] = result1['addresses'][0]['y']

rows[612][14] = result2['addresses'][0]['x']
rows[612][13] = result2['addresses'][0]['y']

rows[1321][14] = result3['addresses'][0]['x']
rows[1321][13] = result3['addresses'][0]['y']

rows[1324][14] = result3['addresses'][0]['x']
rows[1324][13] = result3['addresses'][0]['y']

f = open('cultureFestival.csv', 'w', newline='')
writer = csv.writer(f)
writer.writerows(rows)
f.close()
for i in range(len(rows)):
    if rows[i][13] == '' or rows[i][14] == '':
        print(i+1, rows[i][0])

### 위 코드에서 출력 결과가 없으므로 모든 데이터에 위도와 경도가 입력됐다.
### step3. 데이터가 없을 경우 값을 NULL로 만든다. 또한 내 위치와 문화축제지까지의 거리를 넣을 인덱스를 각 rows의len(rows) 자리에 빈 칸으로 생성해놓는다.

In [22]:
import csv
#파일을 열어서 데이터 수정
f = open('cultureFestival.csv', 'r', newline='')
reader = csv.reader(f)
rows = list(reader)
f.close()

for i in range(len(rows)):
    rows[i].append('')
    for j in range(len(rows[0])):
        if rows[i][j] == '':
            rows[i][j] = 'NULL'

In [24]:
print(rows[0])
print(rows[1])

['축제명', '개최장소', '축제시작일자', '축제종료일자', '축제내용', '주관기관명', '주최기관명', '후원기관명', '전화번호', '홈페이지주소', '관련정보', '소재지도로명주소', '소재지지번주소', '위도', '경도', '데이터기준일자', '제공기관코드', '제공기관명', ' ']
['2022년 순천민주주의 정책 페스티벌', '부읍성남문터광장(예정)', '2022-11-12', '2022-11-12', '시민공론장, 민주주의정책 홍보관, 배움터, 체험 등', '전라남도 순천시청', '전라남도 순천시청', 'NULL', '061-749-5554', 'NULL', '축제일정 변동가능', '전라남도 순천시 중앙로 93', 'NULL', '34.9539016', '127.4831807', '2022-09-19', '4820000', '전라남도 순천시', ' ']


In [25]:
#수정된 데이터를 csv에 다시 작성
f = open('cultureFestival.csv', 'w', newline='')
writer = csv.writer(f)
writer.writerows(rows)
f.close()